## ATENCION !!!!!
Una vez ejecutada la notebook apagar el kernel (ir a menú File --> Stop Kernel). Para restartearlo: menú Kernel --> Restart

In [1]:
## SETEO DE VARIABLES (PRESTAR ATENCION ACÁ)

## SETEO OBLIGATORIO!!!

## CAMBIAR: workspace_id: es el skill-id del skill que quieren probar
workspace_id = '' 

## CAMBIAR: NOMBRE ARCHIVOS CSV A IMPORTAR
## IBM_COS_FILE_KEY: es el archivo CSV que tienen en el bucket y que van a utilizar para las pruebas (deben poner el nombre de su archivo)
IBM_COS_FILE_KEY = 'noEntendimiento.csv'


## CAMBIAR: Si quiero grabar archivo con errores poner True, sino poner False
GRABAR_ARCHIVO = True

## CAMBIAR: si quiero modificar la confianza mínima de intenciones (hoy 0.65)
MIN_CONFIDENCE = 0.60

## AMBIENTE DE LOS SKILLS (valores posibles: "newIR-DEV" (nuevo dev) o "assistant-watson-dev-cognitive" (viejo dev))
ENTORNO = "newIR-DEV" 

## Separador de campos del csv
separator = ","

## IBM_COS_BUCKET: es el Bucket donde se guardan los files de prueba.
##  Lo pueden cambiar si desean crear un nuevo bucket y utilizar ese, sino pueden dejar el que está.
IBM_COS_BUCKET = 'watsonstudio' 

############################################################################################################
## SETEO OPCIONAL (REVISAR SI LAS VARIABLES ESTÁN OK) 
# ENDPOINT_URL: lo dejan como esta ya que es la URL donde contesta el cloud object storage (apunta al de desarrollo)
ENDPOINT_URL = 'https://' # change this based on the region of your cos bucket 
# COS_API_KEY_ID: idem al anterior 
COS_API_KEY_ID = ''
# RESOURCE_INSTANCE_ID: idem al anterior 
RESOURCE_INSTANCE_ID = 'crn:'
# En qué entorno
if ENTORNO == "newIR-DEV":
    url_datacenter = 'https://'
    iam_apikey = ''
else:
    url_datacenter = 'https://'
    iam_apikey = ''


In [2]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context

ModuleNotFoundError: No module named 'project_lib'

In [13]:
from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')

# Skill Testing for Watson Assistant

In [14]:
!pip install --index-url https://pypi.python.org/simple  -U "pip"
!pip install --index-url https://pypi.python.org/simple  -U "assistant-dialog-skill-analysis>=1.2.1"

Looking in indexes: https://pypi.python.org/simple
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://pypi.python.org/simple


DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [15]:
!pip install --index-url https://pypi.python.org/simple  -U "nbconvert == 6.5.0"

Looking in indexes: https://pypi.python.org/simple
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [16]:
# Standard python libraries
import sys, os
import json
import importlib
from collections import Counter

# External python libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import ibm_watson

# Internal python libraries
from assistant_dialog_skill_analysis.utils import skills_util, lang_utils
from assistant_dialog_skill_analysis.highlighting import highlighter
from assistant_dialog_skill_analysis.data_analysis import summary_generator
from assistant_dialog_skill_analysis.data_analysis import divergence_analyzer
from assistant_dialog_skill_analysis.data_analysis import similarity_analyzer
from assistant_dialog_skill_analysis.term_analysis import chi2_analyzer
from assistant_dialog_skill_analysis.term_analysis import keyword_analyzer
from assistant_dialog_skill_analysis.term_analysis import entity_analyzer
from assistant_dialog_skill_analysis.confidence_analysis import confidence_analyzer
from assistant_dialog_skill_analysis.inferencing import inferencer
from assistant_dialog_skill_analysis.experimentation import data_manipulator

[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Table of Contents

1. [Part 1: Training Data Analysis](#part1)<br>
2. [Part 2: Model Analysis](#part2)<br>
    2.1 [Model Analysis](#part2.1)<br>
3. [Part 3: Advanced Analysis](#part3)<br>
    3.1 [Abnormal Confidence Analysis](#part3.1)<br>

# Part 1 : Training Data Analysis

<a id='setup'></a>
## Setup: Access Training Data

Please provide access credentials for an existing dialog skill that you would like to analyze.  
Have your API Key & Workspace ID values handy

## ATENCION !!!!!
Una vez ejecutado apagar el kernel

In [17]:
importlib.reload(skills_util)

# Change Assistant API version if needed
# Find Latest --> https://cloud.ibm.com/docs/services/assistant?topic=assistant-release-notes
API_VERSION = '2023-05-25'

# choose a datacenter to use
datacenters = {
    'dallas'    : (url_datacenter,'https://iam.cloud.ibm.com/identity/token'), 
}
URL, authenticator_url = datacenters['dallas']

# For ICP(IBM Cloud Private), you can disable SSL verification by changing this to True
DISABLE_SSL_VERTIFICATION = False 

#iam_apikey, workspace_id = skills_util.input_credentials()
conversation = skills_util.retrieve_conversation(iam_apikey=iam_apikey,url=URL,api_version=API_VERSION,authenticator_url=authenticator_url)
conversation.set_disable_ssl_verification(DISABLE_SSL_VERTIFICATION)
workspace = skills_util.retrieve_workspace(workspace_id=workspace_id,
                                           conversation=conversation)

Pick the language code correspond to your workspace data:   
*Supported Language codes:* **en, fr, de, es, cs, it, pt**

In [18]:
importlib.reload(lang_utils)
LANGUAGE_CODE="es" # change the language code to work with other languages
lang_util = lang_utils.LanguageUtility(LANGUAGE_CODE)

In [19]:
# Extract user workspace
workspace_pd, workspace_vocabulary = skills_util.extract_workspace_data(workspace, language_util=lang_util)
entity_dict = conversation.list_entities(workspace_id).get_result()
entities_list = [item['entity'] for item in entity_dict['entities']]

<a id='part2'></a>
# Part 2: Model Analysis

Analyze your existing Watson Assistant Dialog Skill with the help of a test set.

## Setup: Upload Test Data
Please upload a test set in csv format. Each line in the file should have only `User_Input<comma>Intent`  

In [20]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

cos_client = ibm_boto3.client(service_name='s3',ibm_api_key_id=COS_API_KEY_ID,ibm_service_instance_id = RESOURCE_INSTANCE_ID,config=Config(signature_version='oauth'),endpoint_url=ENDPOINT_URL)

body = cos_client.get_object(Bucket=IBM_COS_BUCKET,Key=IBM_COS_FILE_KEY)['Body']
# add missing __iter__ method
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
    
test_df = skills_util.process_test_set(body, lang_util, delim=separator, cos=True)
#display(Markdown("### Random Test Sample"))
#display(HTML(test_df.sample(n=min(10,len(test_df))).to_html(index=False)))   

## Evaluate Test Data
These steps can take time if you have a large test set  

<font color=red>**Note**</font>: You will be charged for calls made from this notebook  based on your WA plan 

In [21]:
# Maximum of 5 threads for faster inference
THREAD_NUM = 5
full_results = inferencer.inference(conversation,workspace_id,test_df,max_retries=10,max_thread=THREAD_NUM,verbose=False)

Starting Thread-1Starting Thread-2

Starting Thread-3
Starting Thread-4
Starting Thread-5
Exiting Thread-3
Exiting Thread-1Exiting Thread-5

Exiting Thread-2
Exiting Thread-4
--- Total time: 297.8124432563782 seconds ---


<a id='part2.1'></a>
## 2.1 Model Analysis

### Test Data Overview

In [22]:
importlib.reload(summary_generator)
#summary_generator.generate_summary_statistics(test_df)
summary_generator.show_user_examples_per_intent(test_df)

### Sorted Distribution of User Examples per Intent

,Intent,Number of User Examples
1,IR_noEntendimiento,3475


In [23]:
# agrega la 2da y 3ra intencion y confianza
full_results.insert(4, 'top2_intent', full_results.apply(lambda x: x["top_predicts"][1]["intent"], axis = 1))
full_results.insert(5, 'top2_confidence', full_results.apply(lambda x: x["top_predicts"][1]["confidence"], axis = 1))
#full_results.insert(6, 'top3_intent', full_results.apply(lambda x: x["top_predicts"][2]["intent"], axis = 1))
#full_results.insert(7, 'top3_confidence', full_results.apply(lambda x: x["top_predicts"][2]["confidence"], axis = 1))

# calcula accuracy
importlib.reload(inferencer)
results = full_results[['correct_intent','top_intent','top_confidence','top2_intent','top2_confidence','utterance']]
accuracy = inferencer.calculate_accuracy(results)

# errores
importlib.reload(inferencer)
wrongs_df = inferencer.calculate_mistakes(results)

#predicted correctly with low confidence & predicted incorrectly with high confidence
importlib.reload(confidence_analyzer)
correct_thresh, wrong_thresh = MIN_CONFIDENCE, MIN_CONFIDENCE
correct_with_low_conf_list, incorrect_with_high_conf_list = confidence_analyzer.abnormal_conf(full_results, correct_thresh, wrong_thresh)

# joinea los errores con las de baja confidence
if not wrongs_df.empty and not correct_with_low_conf_list.empty:
    wrongs_df = wrongs_df.append(correct_with_low_conf_list, ignore_index=False)
elif not correct_with_low_conf_list.empty:
    wrongs_df = correct_with_low_conf_list

# calcula accuracy total de errores con correctas intenciones con baja confidence
if len(test_df) > 0:
    accuracy_all = (len(test_df) - len(wrongs_df)) * 100 / len(test_df)
else:
    accuracy_all = 0

In [24]:
wrongs_df

,correct_intent,top_intent,top_confidence,top2_intent,top2_confidence,utterance
Test Example Index,,,,,,
0,IR_noEntendimiento,noMeLlegoLaFactura,0.058383,consultarVisita,0.050430,﻿a donde
1,IR_noEntendimiento,consultarVisita,0.066805,adherirDebitoAutomatico,0.061670,a la iniciativa
2,IR_noEntendimiento,consultarSaldo,0.104953,consultarVisita,0.104088,a entenderas mi pregunta no tendras
3,IR_noEntendimiento,consultarVisita,0.068945,consultarSaldo,0.062214,a eso cambie de opinion un besito de agua gracias
4,IR_noEntendimiento,consultarVisita,0.042895,renovarDatos,0.040856,a dos
...,...,...,...,...,...,...
3470,IR_noEntendimiento,meLlegoMas,0.071566,hablarConRepresentante,0.070693,yo pongo
3471,IR_noEntendimiento,renovarDatos,0.432499,consultaDatosDisponibles,0.309949,yo quiero preguntar por los gigas que me van a...
3472,IR_noEntendimiento,hablarConRepresentante,0.561681,renovarDatos,0.227190,yo quiero hacer varias consultas a una persona...


### Determine Overall Accuracy on Test Set

#### (Mirar Bien Cuánto Da el % de "Accuracy on Test Data" en la sigte celda)

In [25]:
display(Markdown("### ==&gt; Accuracy on Test Data: {} % ".format(accuracy)))
display(Markdown("## ==&gt; Accuracy on Test Data including Correctly Predicted with <u>Low Confidence</u>: <u>{}</u> % ".format(round(accuracy_all,2))))

### ==&gt; Accuracy on Test Data: 0.0 % 

## ==&gt; Accuracy on Test Data including Correctly Predicted with <u>Low Confidence</u>: <u>0.0</u> % 

### Error Analysis

This section gives the user an overview of the errors made by the intent classifier on the test set  

### En la sigte celda se ven los utterances con errónea predicción e incluyen los que sí tuvieron buena predicción pero baja confianza

In [26]:
display(Markdown("### Intent Detection Mistakes"))
display(Markdown("Number of Test Errors (including correctly predicted with low confidence): {}".format(len(wrongs_df))))
#project.save_data(file_name = "PRUEBA.csv",data = df_logs.to_csv(index=False))
with pd.option_context('max_colwidth', 250):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
        if not wrongs_df.empty:
            display(wrongs_df)

### Intent Detection Mistakes

Number of Test Errors (including correctly predicted with low confidence): 3475

,correct_intent,top_intent,top_confidence,top2_intent,top2_confidence,utterance
Test Example Index,,,,,,
0,IR_noEntendimiento,noMeLlegoLaFactura,0.058383,consultarVisita,0.050430,﻿a donde
1,IR_noEntendimiento,consultarVisita,0.066805,adherirDebitoAutomatico,0.061670,a la iniciativa
2,IR_noEntendimiento,consultarSaldo,0.104953,consultarVisita,0.104088,a entenderas mi pregunta no tendras
3,IR_noEntendimiento,consultarVisita,0.068945,consultarSaldo,0.062214,a eso cambie de opinion un besito de agua gracias
4,IR_noEntendimiento,consultarVisita,0.042895,renovarDatos,0.040856,a dos
5,IR_noEntendimiento,consultarVisita,0.064586,migracionAPrepago,0.063621,a manana
6,IR_noEntendimiento,portabilidad,0.035195,consultarVisita,0.032419,a largo
7,IR_noEntendimiento,consultarVisita,0.104684,consultarSaldo,0.056326,a mi a mi hidratativo a mi hidratativo
8,IR_noEntendimiento,consultarVisita,0.067869,cambioTitularidad,0.066579,a mi creativo


In [27]:
# Grabar archivo? 
if GRABAR_ARCHIVO == True and not wrongs_df.empty:
    extension = '.csv'
    archivo = IBM_COS_FILE_KEY[:IBM_COS_FILE_KEY.rfind(extension.lower())] + '__Errors.csv'
    # limpia basura
    archivo = archivo.replace("(", ".")
    archivo = archivo.replace(")", ".")
    # graba
    try:
        #project.save_data(archivo, wrongs_df.to_csv(mode='w',index=True))
        project.save_data(data=wrongs_df.to_csv(mode='w',index=True),file_name=archivo,overwrite=True)
    except Exception as e:
        strerr = str(e)
        if strerr.rfind('already exists') > 0:
            display(Markdown('### ERROR: el archivo '+archivo+' ya existe y no se puede sobreescribir. <br><br><i>Nota: Intentá eliminar el archivo en el storage del Cloudpak y volvé a ejecutar SOLO esta celda para que se grabe el archivo.</i>'))
        else:
            print(Exception, e)
    else:
        display(Markdown("### Archivo de Errores grabado como: " + archivo))
        print ('Nota: El archivo se grabó como Asset en Cloudpak, no en el bucket. Después de descargarlo, por favor eliminalo del repositorio.')
    # graba en Bucket (no funciona)
    #try:
    #    res=cos_client.upload_file(Filename=archivo, Bucket=IBM_COS_BUCKET, Key=IBM_COS_FILE_KEY)
    #except Exception as e:
    #    print(Exception, e)
    #else:
    #    print('Archivo de Errores grabado como: ' + archivo)


### Archivo de Errores grabado como: noEntendimiento__Errors.csv

Nota: El archivo se grabó como Asset en Cloudpak, no en el bucket. Después de descargarlo, por favor eliminalo del repositorio.


<a id='part3'></a>
## Part 3: Advanced Analysis

<a id='part3.1'></a>
## 3.1 Abnormal Confidence Analysis
Every test utterance is classified as a specific intent with a specific confidence by the WA intent classifier. It is expected that model would be confident when correctly predicting examples and not highly confident when incorrectly predicting examples. 

But often this is not true. This may suggest there are anomalies in the design. Examples that are predicted correctly with low confidence and the examples that are predicted incorrectly with high confidence are thus cases which need to be reviewed.

In [28]:
if len(correct_with_low_conf_list) > 0:
    display(Markdown("### Examples correctedly predicted with low confidence"))
    display(Markdown("(Estos resultados ya se encuentran en el dataset anterior: Intent Detection Mistakes)"))
    with pd.option_context('max_colwidth', 250):
        display(HTML(correct_with_low_conf_list.to_html(index=False)))

In [29]:
if len(incorrect_with_high_conf_list) > 0:
    display(Markdown("### Examples incorrectedly predicted with high confidence"))
    display(Markdown("(Estos resultados ya se encuentran en el dataset anterior: Intent Detection Mistakes)"))
    with pd.option_context('max_colwidth', 250):
        display(HTML(incorrect_with_high_conf_list.to_html(index=False)))

### Examples incorrectedly predicted with high confidence

(Estos resultados ya se encuentran en el dataset anterior: Intent Detection Mistakes)

correct_intent,utterance,top1_prediction,top2_prediction,top3_prediction
IR_noEntendimiento,a mi nombre,cambioTitularidad (0.985),consultarSaldo (0.297),adherirDebitoAutomatico (0.279)
IR_noEntendimiento,a ver yo pague personal el xx de octubre y ya me estan diciendo que debo xxxx estan locos que es esto,pagueDebo (0.705),informarPago (0.195),consultaCicloFacturacion (0.17)
IR_noEntendimiento,abono mensual,cambioPlan (0.685),meLlegoMas (0.335),comoPago (0.233)
IR_noEntendimiento,acaba de pehuajo en gonzalez xxx no hay canal desde las xx horas no hay canal,noTengoCable (0.601),noTengoInternet (0.326),consultarVisita (0.323)
IR_noEntendimiento,activar pack,packs (0.985),reclamoRecarga (0.308),suspensionPorRobo (0.274)
IR_noEntendimiento,asesor,hablarConRepresentante (0.752),problemaTecnico (0.444),comoPago (0.339)
IR_noEntendimiento,asesor de atencion al cliente,hablarConRepresentante (0.9),meLlegoMas (0.242),cambioTitularidad (0.241)
IR_noEntendimiento,asesor humano,hablarConRepresentante (0.837),problemaTecnico (0.506),comoPago (0.432)
IR_noEntendimiento,asistente por favor,hablarConRepresentante (0.846),cambioPlan (0.292),estadoEquipo (0.282)
IR_noEntendimiento,atencion al cliente,hablarConRepresentante (1),adherirDebitoAutomatico (0),cambiarNumero (0)
